In [44]:
import requests
import os

global input_path 
global output_path



def send_image_to_server(image_path, output_path):
    url = "http://127.0.0.1:8000/process_image"  # 服务地址
    with open(image_path, "rb") as file:
        files = {"image": file}
        data = {"out_file": output_path}  # 发送字符串
        response = requests.post(url, files=files, data=data)
        if response.status_code == 200:
            return f"Server Response:{response.json()}"
        else:
            return f"Error:{response.json()}"


In [45]:

from autogen_core.base import CancellationToken
from autogen_core.components.tools import PythonCodeExecutionTool
from autogen_ext.code_executors import DockerCommandLineCodeExecutor

# Create the tool.
code_executor = DockerCommandLineCodeExecutor()
await code_executor.start()
code_execution_tool = PythonCodeExecutionTool(code_executor)
cancellation_token = CancellationToken()

from langchain_core.messages import AIMessage
from langchain_core.tools import tool

from langgraph.prebuilt import ToolNode

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

@tool
async def code_executor(code: str):
    """excute code in the python environment, return result"""
    result = await code_execution_tool.run_json({"code": code}, cancellation_token)

    return code_execution_tool.return_value_as_string(result)

@tool
def detect_defects():
    """detect defects"""
    text = "The defect types are two: impurity and void."
    # label = "void :[ 298.8452,  525.0196,  580.6752,  998.5887]"
    format = "The format of the JSON is: 'defect name : defect coordinate [x1, y1, x2, y2]'. These are the detected defects:"
    
    res = send_image_to_server(input_path,output_path)
    return text+format+res

tools = [code_executor, detect_defects]
tool_node = ToolNode(tools)


from typing import Literal

from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    model="GLM-4-plus",
    openai_api_key="your api key",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

model_with_tools = llm.bind_tools(tools)


from typing import Literal

from langgraph.graph import StateGraph, MessagesState, START, END


def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END


def call_model(state: MessagesState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}


workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")

app = workflow.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "2"}}
"""
{"messages": [("human", "excute code print('Hello, world! I love you')")]}, stream_mode="values"
"""
# await app.ainvoke({"messages": [("human", "detect defects and then calculate the defects' area. once you use the code tool, your code you provide should print the code execution result.")]})


'\n{"messages": [("human", "excute code print(\'Hello, world! I love you\')")]}, stream_mode="values"\n'

In [46]:
import gradio as gr
from PIL import Image, ImageDraw, ImageFont
import os


import requests

from langchain.schema import AIMessage, HumanMessage


# 主函数
async def process_images_with_watermark(message, history):
    # uploaded_images = message["files"]
    # processed_images = []
    response=[]

    if message.get("files"):
        for file in message["files"]:
            
            global input_path
            global output_path
            input_dir = "D:\\llm\\langchain\\缺陷检测\\input"
            output_dir = "D:\\llm\\langchain\\缺陷检测\\output"
            os.makedirs(input_dir, exist_ok=True)  # 创建临时目录

            input_path = os.path.join(input_dir, os.path.basename(file))
            output_path = os.path.join(output_dir, os.path.basename(file))
            

            image = Image.open(file).convert("RGB")
            image.save(input_path)

            input_message = HumanMessage(content="System: For math-related problems, please use the code tool. Ensure that the provided code prints the execution result. \n \n "+message['text'])
            res = await app.ainvoke({"messages": input_message },config)
            # res = send_image_to_server(input_path,output_path)
            
            response.append(res["messages"][-1].content)
            # watermarked_image.save(output_path)
            response.append(gr.Image(output_path))
    else:

        input_message = HumanMessage(content="System: For math-related problems, please use the code tool. Ensure that the provided code prints the execution result. \n \n "+message['text'])
        res = await app.ainvoke({"messages": input_message },config)
        response.append(res["messages"][-1].content)
    return response
    

# 创建 Gradio 接口
demo = gr.ChatInterface(
    fn=process_images_with_watermark,
    type="messages",
    examples=[
        {"text": "No files", "files": []}
    ],
    multimodal=True,
    textbox=gr.MultimodalTextbox(file_count="multiple", file_types=["image"], sources=["upload"])
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.
